# ktrainから使うBERT

ktrainはKerasのラッパーであり、ニューラルネットワークを簡単に学習できます。わずか数行のコードを書くだけでモデルを構築し、最適な学習率を推定し、テキストの読み込みや前処理を行い、モデルを学習できます。

- https://github.com/amaiya/ktrain

## 準備

### パッケージのインストール

In [1]:
!pip3 install ktrain==0.27.3

     |████████████████████████████████| 25.3 MB 96 kB/s 
     |████████████████████████████████| 6.8 MB 35.9 MB/s 
     |████████████████████████████████| 981 kB 31.0 MB/s 
     |████████████████████████████████| 263 kB 43.0 MB/s 
     |████████████████████████████████| 1.9 MB 31.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.5 MB/s 
     |████████████████████████████████| 468 kB 46.7 MB/s 
     |████████████████████████████████| 895 kB 36.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.27.3-py3-none-any.whl size=25283131 sha256=fa918db910b5deb03c38c50d084471fb1aeda8b7a5fa19c976db817bef8d54f1
  Stored in directory: /root/.cache/pip/wheels/3c/ef/e1/7da805d8a5944e8a3ac0553831d832c00b1800b848939849cb
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=3b9cc4b3bc1181000cbb32fdda05a3b2e6fabe07627d1e6f8816dd2ecfec53bb
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff0

### インポート

In [2]:
import ktrain
from ktrain import text

### データセットのダウンロード

データセットは以前にも使ったIMDB映画レビューデータセットを使います。

In [3]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz",
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True,
)

84140032/84125825 [==============================] - 6s 0us/step


In [4]:
%cd /root/.keras/datasets/aclImdb
!ls

/root/.keras/datasets/aclImdb
imdbEr.txt  imdb.vocab	README	test  train


読み込むときのために、フォルダの構造を確認しておきましょう。
`train`と`test`の下に`pos`と`neg`フォルダがあり、その中にプレーンテキストでレビューが格納されています。おおよそ、以下の構造になります。

```bash
├── datadir
│   ├── train
│   │   ├── pos       # 肯定的なレビューのテキストを含むフォルダ
│   │   └── neg       # 否定的なレビューのテキストを含むフォルダ
│   └── test 
│        ├── pos       # 肯定的なレビューのテキストを含むフォルダ
│        └── neg       # 否定的なレビューのテキストを含むフォルダ
```

In [22]:
!ls -F train

labeledBow.feat  pos/	 unsupBow.feat	urls_pos.txt
neg/		 unsup/  urls_neg.txt	urls_unsup.txt


In [5]:
# set path to dataset
import os.path

dataset = "/root/.keras/datasets/aclImdb"
IMDB_DATADIR = os.path.join(os.path.dirname(dataset), "aclImdb")
print(IMDB_DATADIR)

/root/.keras/datasets/aclImdb


## 前処理

`texts_from_folder`関数を使うと、学習用と検証用のデータを指定したフォルダから読み込み、BERTに必要な形式へ自動的に変換してくれます。この関数は、以下のようなフォルダ構造でコーパスが格納されていることを仮定しています。

```bash
├── datadir
│   ├── train
│   │   ├── class0       # folder containing documents of class 0
│   │   ├── class1       # folder containing documents of class 1
│   │   ├── class2       # folder containing documents of class 2
│   │   └── classN       # folder containing documents of class N
│   └── test 
│       ├── class0       # folder containing documents of class 0
│       ├── class1       # folder containing documents of class 1
│       ├── class2       # folder containing documents of class 2
│       └── classN       # folder containing documents of class N
```

各サブフォルダは、プレーンテキスト形式で文書を格納していなければなりません。今回の場合、サブフォルダ名は`pos`と`neg`なので、それを`classes`引数に指定します。

In [11]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(
    IMDB_DATADIR,
    maxlen=128,
    preprocess_mode="bert",
    train_test_names=["train", "test"],
    classes=["pos", "neg"],
)

detected encoding: utf-8
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## 事前学習済みBERTの読み込み

まずは、`text_classifier`関数を使って、テキスト分類モデルを構築します。今回はBERTを使いたいため、関数の第一引数に`bert`と渡しています。それ以外に以下の文字列を渡せます。

- fasttext: FastTextモデル
- nbsvm: NBSVM（ナイーブベイズとSVMの組み合わせ）
- logreg: 埋め込み層を使ったロジスティック回帰
- bigru: 事前学習済み単語埋め込みを使った双方向GRU
- distilbert: DistilBertモデル

その後、`get_learner`関数を呼び出して、モデルを学習するための`Learner`インスタンスを作成します。

In [14]:
model = text.text_classifier("bert", (x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(
    model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=16
)

Is Multi-Label? False
maxlen is 128
done.


## モデルの学習

ここまで来たら、あとは`fit_onecycle`メソッドを呼び出して、モデルを学習します。
第1引数に学習率、第２引数にエポック数を指定しています。

In [15]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
1563/1563 [==============================] - 1805s 1s/step - loss: 0.3853 - accuracy: 0.8230 - val_loss: 0.2926 - val_accuracy: 0.8734
Epoch 2/4
1563/1563 [==============================] - 1752s 1s/step - loss: 0.2580 - accuracy: 0.8931 - val_loss: 0.2750 - val_accuracy: 0.8824
Epoch 3/4
1563/1563 [==============================] - 1751s 1s/step - loss: 0.1463 - accuracy: 0.9458 - val_loss: 0.3092 - val_accuracy: 0.8802
Epoch 4/4
1563/1563 [==============================] - 1750s 1s/step - loss: 0.0393 - accuracy: 0.9885 - val_loss: 0.4177 - val_accuracy: 0.8864
